**Creating a 3D array of size (2, 9, 5)**
```python
array_3d = np.array([
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [41, 42, 43, 44, 45]
    ],
    [
        [46, 47, 48, 49, 50],
        [81, 82, 83, 84, 85],
        [86, 87, 88, 89, 90]
    ]
])
```

**Get flatten array from index [:,:,1]**
```python
array_3d[:,:,1].flatten() 
```

[ 2  7 42 47 82 87]

=**Get flatten array from index [:,1,:]**
```python
array_3d[:,1,:].flatten()
```

[ 6  7  8  9 10 81 82 83 84 85]

**Get flatten array from index [1,:,:]**
```python
array_3d[1,:,:].flatten()
```

[46 47 48 49 50 81 82 83 84 85 86 87 88 89 90]


In [1]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
import Jimp from "jimp";
import { Niivue, NVImage } from "@niivue/niivue";
import { cv } from "opencv-wasm";


In [2]:
// in kernel, The configuration runs using node, and not in the browser,
// which is why window is not defined
// we need to define it to make the code work

// global.window = { navigator: { userAgent: 'node.js' } };

import crypto from "node:crypto";
global.crypto ??= crypto;
const volume = await NVImage.loadFromUrl({url:"https://niivue.github.io/niivue-demo-images/mni152.nii.gz"})
console.log(volume.img)

// import fs from 'fs';

// const writeStream = fs.createWriteStream('file.txt');
// const pathName = writeStream.path;
  
// // write each value of the array on the file breaking line
// volume.img.forEach(value => writeStream.write(`${value}\n`));

// // the finish event is emitted when all data has been flushed from the stream
// writeStream.on('finish', () => {
//    console.log(`wrote all the array data to file ${pathName}`);
// });

// // handle the errors on the write process
// writeStream.on('error', (err) => {
//     console.error(`There is an error writing the file ${pathName} => ${err}`)
// });

// // close the stream
// writeStream.end();

Uint8Array(11393280) [
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0,
  ... 11393180 more items
]


In [3]:
function getMax(arr) {
    let len = arr.length;
    let max = -Infinity;

    while (len--) {
        max = arr[len] > max ? arr[len] : max;
    }
    return max;
}

function getMin(arr) {
    let len = arr.length;
    let min = Infinity;

    while (len--) {
        min = arr[len] < min ? arr[len] : min;
    }
    return min;
}

In [4]:
function normalizeArray(
    array: Float32Array | Uint8Array,
    max: number
  ): Float32Array {
    let normalizedArray = new Float32Array(array.length);
    for (let i = 0; i < array.length; i++) {
      normalizedArray[i] = array[i] / max;
    }
    return normalizedArray;
  }
  
  function stackSliceToRGB(
    buffer: Float32Array | Uint8Array,
  ): Float32Array {
    let bufferLength = buffer.length,
      result = new Float32Array(bufferLength * 3);
  
    for (let i = 0; i < bufferLength; i++) {
      result[3 * i] = buffer[i];
      result[3 * i + 1] = buffer[i];
      result[3 * i + 2] = buffer[i];
    }
    return result;
  }
  
  

In [12]:
const resize = async () => {
    // let jimpSrc = await Jimp.read('/data/Projects/deepsyence/notebook/niivue.png');

    // var src = cv.matFromImageData(jimpSrc.bitmap);
    var image4channel = addChannel(inputImage.data);
    var src = cv.matFromArray(inputImage.size().width, inputImage.size().height, cv.CV_8UC4, image4channel);
    console.log(src.size(), src.type(), src.data);
    let dst = new cv.Mat();
    cv.resize(src, dst, { width: 100, height: 100 }, cv.INTER_CUBIC);
    console.log(dst.size(), dst.type(), dst.data);
    new Jimp({
        width: dst.cols,
        height: dst.rows,
        data: Buffer.from(dst.data)
    })
        .write('/data/Projects/deepsyence/notebook/niivue_resize.png');
}
resize();

5:36 - Cannot find name 'inputImage'.
6:31 - Cannot find name 'inputImage'.
6:56 - Cannot find name 'inputImage'.


In [5]:
const preprocessVolume = (image) => {
    try {
      const imageRAS = image.img2RAS();
      // console.log("image.calMax, image.calMin ", image.global_max, image.global_min);
      const maxVal = getMax(imageRAS);
      const normalizedArray = normalizeArray(imageRAS, maxVal);
      console.log("normalizedArray ", maxVal, normalizedArray.reduce((a, b) => a + b, 0));
      return normalizedArray;
    } catch (e) {
      console.log(`failed to preprocess volume: ${e}. `);
      throw Error(`failed to preprocess volume: ${e}. `);
    }
  }

const preprocess = (slice2D: Float32Array, sliceId: number, width: number, height: number): Float32Array => {
    try {
      // console.log("imageArray ", slice2D);
      let image3Channels: Float32Array;
      const sliceArray = slice2D.slice(
        width * height * sliceId,
        width * height * (sliceId + 1),
      );
      console.log("sliceArray ", sliceArray.reduce((a, b) => a + b, 0));
  
      image3Channels = stackSliceToRGB(sliceArray);
      console.log("image3Channels ", image3Channels, image3Channels.reduce((a, b) => a + b, 0));
      return image3Channels;
    } catch (e) {
      console.log(`failed to inference ONNX model: ${e}. `);
      throw Error(`failed to inference ONNX model: ${e}. `);
    }
};


In [6]:
function addChannel(buffer: Float32Array | Uint8Array): Uint8Array {
  let bufferLength = buffer.length,
    result = new Uint8Array((bufferLength / 3) * 4);

  for (let i = 0; i < bufferLength; i += 3) {
    result[(4 * i) / 3] = buffer[i] * 1;
    result[(4 * i) / 3 + 1] = buffer[i + 1] * 1;
    result[(4 * i) / 3 + 2] = buffer[i + 2] * 1;
    result[(4 * i) / 3 + 3] = 255;
  }
  return result;
}

In [7]:
function resizeLongesSide (image: any, target_length: number) {
    let dstResized = new cv.Mat();
    let image4Channels;
    let oldw = image.size().width
    let oldh = image.size().height
    console.log("oldh, oldw ", oldh, oldw, image.size());
    let scale = target_length * 1.0 / Math.max(oldh, oldw)
    let newh = oldh * scale
    let neww = oldw * scale
    console.log("target_size ", newh, neww, new cv.Size(newh, neww));
    if (image.type() !== cv.CV_8UC4) {
        image4Channels = addChannel(image.data);
    }
    var src = cv.matFromArray(oldw, oldh, cv.CV_8UC4, image4Channels);
    
    cv.resize(src, dstResized, new cv.Size(newh, neww), 0, 0, cv.INTER_CUBIC);
    console.log("dstResized ", dstResized.size(), dstResized.type());
    new Jimp({
        width: dstResized.cols,
        height: dstResized.rows,
        data: Buffer.from(dstResized.data)
    })
        .write('/data/Projects/deepsyence/notebook/niivue_resize_1.png');
    return dstResized;
}

function pad(image: any, target_size: number) {
    let dstPadded = new cv.Mat();
    let image4Channels;
    let h = image.size().height
    let w = image.size().width
    let padh = target_size - h
    let padw = target_size - w
    console.log("padh, padw ", padh, padw);
    if (image.type() !== cv.CV_8UC4) {
        image4Channels = addChannel(image.data);
    }
    //  copyMakeBorder( src, dst, top, bottom, left, right, borderType, value );
    cv.copyMakeBorder(image, dstPadded, 0, padh, 0, padw, cv.BORDER_CONSTANT);
    new Jimp({
        width: dstPadded.cols,
        height: dstPadded.rows,
        data: Buffer.from(dstPadded.data)
    })
        .write('/data/Projects/deepsyence/notebook/niivue_padded.png');
    return dstPadded;
}

In [8]:
function imagedataToImage(imagedata: Float32Array | Uint8Array, dims: number[]) {
    let inputImage;
    // if (dims[0]*dims[1] !== imagedata.length) {
    //     throw new Error("Image data size does not match the dimensions");
    // }

      inputImage = new Uint8Array(dims[0]*dims[1]*3);
      for (let i=0; i<imagedata.length; i++) {
        inputImage[i] = imagedata[i]*255;
    }

    let dst = cv.matFromArray(dims[0], dims[1], cv.CV_8UC3, inputImage);

    // console.inputTensor.dims[3log(dst.data.slice(600, 610));
    // console.log(dst);
    new Jimp({
        width: dims[0],
        height: dims[1],
        data: Buffer.from(dst.data)
    }).write('./niivue.png');
    return dst;
}


In [9]:
function nvVolumetoSlice(volume, dims, fov, index): any {
    let sliceArray;
    switch (fov) {
        // sagittal dimsRAS[1], coronal dimsRAS[2], axial dimsRAS[3]
        
        // 1. if axial, slice the array from volume
        case "axial":
            sliceArray = volume.slice(
                dims[0] * dims[1] * index,
                dims[0] * dims[1] * (index + 1),
            );
            console.log("axial sliceArray",dims[1], dims[2], index, sliceArray.length);
        // 2. if coronal, 
        case "coronal":
        // 3. if sagittal,
        case "sagittal":
    }
    return sliceArray;
    
}

function filterAlphaChannel(
    image: Array<number>,
): Float32Array {
    // 4. convert to float32
    let i,
      l = image.length; // length, we need this for the loop
    console.log("image.length", image.length);
    // create the Float32Array size 3 * 224 * 224 for these dimensions output
    const float32Data = new Float32Array((image.length * 3) / 4);
    for (i = 0; i < l; i += 4) {
      float32Data[(3 * i) / 4] = image[i]; // convert to float
      float32Data[(3 * i) / 4 + 1] = image[i + 1]; // convert to float
      float32Data[(3 * i) / 4 + 2] = image[i + 2]; // convert to float
      // skip image[i + 3] to filter out the alpha channel
    }
    // console.log("float32Data", float32Data)
    return float32Data;
}

function nvSliceToTensor(nvSlice, dims): any {
    console.log("nvSlice", nvSlice.length);
    // 1. preprocess
    let image3Channels = stackSliceToRGB(nvSlice);
    // 2. convert to float32
    console.log("image3Channels", image3Channels.length);
    // 3. tensor
    const inputTensor = new Tensor("float32", image3Channels, dims);
    return inputTensor;
}

function imageDataToTensor(data, dims, max): any {
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B]: number[][] = [[], [], []];
    const channels = (data.length / (dims[3]*dims[2]*3)) % 4 ? 3 : 4;
    console.log("channels", data.length, channels);
    for (let i = 0; i < data.length; i += channels) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
    const transposedData = R.concat(G).concat(B);
  
    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(dims[3]*dims[1]*dims[2]); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] ; // divide by max value to convert to float
    }
  
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
  }

function arrayToTensor(array, dims): any {
    let float32Data = new Float32Array(array.length*3);
    for (let i = 0; i < array.length; i++) {
        float32Data[i] = array[i];
        float32Data[i + array.length] = array[i];
        float32Data[i + 2 * array.length] = array[i];
    }
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
}

In [10]:
// LiteMedSAM
const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
let nvVolume = preprocessVolume(volume);
let nvSlice = nvVolumetoSlice(nvVolume, dims, "axial", 90);
let inputTensor = nvSliceToTensor(nvSlice, [1, 3, dims[0], dims[1]]);

let inputImage = imagedataToImage(inputTensor.data, [dims[0], dims[1]]);
let resizedImage = resizeLongesSide(inputImage, 256);
let paddedImage = pad(resizedImage, 256);
let filteredImage = filterAlphaChannel(paddedImage.data);
console.log("paddedImage ", paddedImage.size(), paddedImage.type(), paddedImage.data);
const maxVal = getMax(filteredImage);
let normalizedImage = normalizeArray(filteredImage, maxVal);
// let stackImage = stackSliceToRGB(normalizedImage);
let transposedTensor = imageDataToTensor(normalizedImage, [1, 3, paddedImage.size().width, paddedImage.size().height], maxVal);


normalizedArray  252 3269611.783972291
axial sliceArray 256 215 90 52992
nvSlice 52992
image3Channels 158976
oldh, oldw  207 256 { width: 256, height: 207 }
target_size  207 256 Size { width: 207, height: 256 }
dstResized  { width: 207, height: 256 } 24
padh, padw  0 49
image.length 262144
paddedImage  { width: 256, height: 256 } 24 Uint8Array(262144) [
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255, 0, 0, 0, 255, 0, 0, 0, 255,
  0, 0, 0, 255,
  ... 262044 more items
]
channels 196608 3


In [21]:
getMax(transposedTensor.data);

1


In [28]:
transposedTensor.data.reduce((a, b) => a + b, 0);
paddedImage.data.reduce((a, b) => a + b, 0);
resizedImage.data.reduce((a, b) => a + b, 0);
// inputImage.data.reduce((a, b) => a + b, 0);
// nvSlice.reduce((a, b) => a + b, 0);

33123738


In [62]:
new Jimp({
    width: resizedImage.cols,
    height: resizedImage.rows,
    data: Buffer.from(resizedImage.data)
}).write('./padded.png');

Jimp {
  _events: [Object: null prototype] {},
  _eventsCount: 0,
  _maxListeners: undefined,
  bitmap: {
    data: <Buffer 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 00 ff 00 00 ... 211918 more bytes>,
    width: 207,
    height: 256
  },
  _background: 0,
  _originalMime: 'image/png',
  _exif: null,
  _rgba: true,
  writeAsync: [Function (anonymous)],
  getBase64Async: [Function (anonymous)],
  getBuffer: [Function: getBuffer],
  getBufferAsync: [Function: getBufferAsync],
  getPixelColour: [Function: getPixelColor],
  setPixelColour: [Function: setPixelColor],
  [Symbol(kCapture)]: false
}


In [8]:
// EfficientSAM
const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
let nvVolume = preprocessVolume(volume);
let nvSlice = nvVolumetoSlice(nvVolume, dims, "axial", 90);
let inputTensor = nvSliceToTensor(nvSlice, [1, 3, dims[0], dims[1]]);
const maxVal = getMax(volume.img2RAS());
let transposedTensor = imageDataToTensor(inputTensor.data, [1, 3, dims[1], dims[0]], maxVal);

normalizedArray  3269611.783972291
axial sliceArray 256 215 90 52992
nvSlice 52992


image3Channels 158976


In [11]:
inputTensor.data.slice(16000,16007)

Float32Array(7) [
  0.4246031641960144,
  0.4246031641960144,
  0.2936508059501648,
  0.2936508059501648,
  0.2936508059501648,
  0.1626984179019928,
  0.1626984179019928
]


In [80]:
// const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
// // let nvVolume = preprocessVolume(volume);
// let nvSlice = nvVolumetoSlice(volume.img2RAS(), dims, "axial", 90);
// // let inputImage = nvSliceToImage(nvSlice);
// const maxVal = getMax(volume.img2RAS());
// let inputTensor = arrayToTensor(nvSlice, [1, 3, dims[0], dims[1]], maxVal);

axial sliceArray 256 215 90 52992


In [33]:
imagedataToImage(inputTensor.data, [dims[0], dims[1]]);

Mat {}


In [30]:
console.log(transposedTensor.data.reduce((a,b) => a+b,0), transposedTensor.dims, getMax(transposedTensor.data), getMin(transposedTensor.data));

console.log(volume.img2RAS().reduce((a,b) => a+b, 0), getMax(volume.img), getMin(volume.img), volume.global_max, volume.global_min);

77127.94049503142 [ 1, 3, 256, 207 ] 0.932539701461792 0
823942169 252 0 80 40


In [24]:
console.log(inputTensor.data.reduce((a,b) => a+b,0), inputTensor.dims, getMax(inputTensor.data), getMin(inputTensor.data));

console.log(volume.img2RAS().reduce((a,b) => a+b, 0), getMax(volume.img), getMin(volume.img), volume.global_max, volume.global_min);

77127.94049503142 [ 1, 3, 256, 207 ] 0.932539701461792 0
823942169 252 0 80 40


In [11]:
global.self = global;
env.wasm.numThreads = 1;

// LiteMedSAM
const encoderSession = await InferenceSession.create('../public/model/lite_medsam_encoder_optimized.onnx');
const decoderSession = await InferenceSession.create('../public/model/litemedsam_decoder_nomaskinput.opt.onnx');

// Efficient SAM
// const encoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_encoder.onnx');
// const decoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_decoder.onnx');


In [12]:
async function runEncoder(model, preprocessedData): Promise<[Tensor, number]> {
    const start = new Date();
    try {
      const feeds: Record<string, Tensor> = {};
      feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

async function runDecoder(model, embedding, inputPoints, inputLabels, oriSize): Promise<[Tensor, number]> {
  const decoderStart = new Date();
  try {
    const decoderFeeds: Record<string, Tensor> = {};
    console.log("model.inputNames", model.inputNames)

    const maskInput = new Tensor(
      "float32",
      new Float32Array(256 * 256),
      [1, 1, 256, 256]
    );
    // There is no previous mask, so default to 0
    const hasMaskInput = new Tensor("float32", [0]);

    decoderFeeds[model.inputNames[0]] = embedding;
    decoderFeeds[model.inputNames[1]] = inputPoints;
    decoderFeeds[model.inputNames[2]] = inputLabels;
    // decoderFeeds[model.inputNames[3]] = maskInput;
    // decoderFeeds[model.inputNames[4]] = hasMaskInput;
    decoderFeeds[model.inputNames[3]] = oriSize;

    // feeds[model.inputNames[1]] = new Tensor("float32", inputPoints, [1, 1, inputPoints.length/2, 2]);
    // feeds[model.inputNames[2]] = new Tensor("float32", new Array(inputPoints.length/2).fill(1), inputPoints.length/2);
    // feeds[model.inputNames[3]] = new Tensor("int64", oriSize);
    const outputData = await model.run(decoderFeeds);
    const decoderEnd = new Date();
    const decoderInferenceTime = (decoderEnd.getTime() - decoderStart.getTime());
    const decoderOutput = outputData[model.outputNames[0]];
    return [decoderOutput, decoderInferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

In [13]:
const [embedding, encoderTime] =  await runEncoder(encoderSession, transposedTensor);
var encoderOutput = embedding.data;
var inferenceTime = encoderTime;
// var results = softmax(Array.prototype.slice.call(output))

In [131]:
console.log((encoderOutput as Float32Array).reduce((a,b) => a + b, 0), inferenceTime, getMax(encoderOutput), getMin(encoderOutput));

-2369.545148893376 11578 0.7205691337585449 -0.9109674096107483


In [15]:
const scale = 256 / Math.max(dims[0], dims[1]);

In [16]:
let points = new Float32Array([134*scale, 150*scale])
let labels = new Float32Array(points.length/2).fill(1)
let boxTensor: Tensor = new Tensor("float32", new Float32Array([0,0,205,254]), [1, 2, 2]);
// let pointsTensor: Tensor = new Tensor("float32", points, [1, 1, 2]);
let labelsTensor: Tensor = new Tensor("float32", new Float32Array([2,3]), [1,2]);
let originalSize: Tensor = new Tensor("int64", [dims[1], dims[0]], [2]);
const [mask, decoderTime] = await runDecoder(decoderSession, embedding, boxTensor, labelsTensor, originalSize);
var decoderOutput = mask.data;
var inferenceTime = decoderTime;

model.inputNames [
  'image_embeddings',
  'batched_point_coords',
  'batched_point_labels',
  'orig_im_size'
]


In [17]:
let maskImage = new Uint8Array(dims[0]*dims[1]);
let decoderOutputSlice = (decoderOutput as Float32Array).slice(0,dims[0]*dims[1]);
for (let i=0; i<decoderOutputSlice.length; i++) {
    if (decoderOutputSlice[i] >= 0) {
        maskImage[i] = 1;
        // maskImage[i+1] = 1;
        // maskImage[i+2] = 1;
    } else {
        maskImage[i] = 0;
        // maskImage[i+1] = 0;
        // maskImage[i+2] = 0;
    }
}
// let transposedMask = transposeChannel(maskImage, mask.dims)


In [94]:
decoderOutput.length

52992


In [18]:
maskImage.reduce((a,b) => a+b, 0)
// getMax(maskImage)

// maskImage.length

24473


In [19]:
let stack_mask = stackSliceToRGB(maskImage)

In [20]:
imagedataToImage((stack_mask), [dims[0], dims[1]]);

Mat {}


In [57]:
function sigmoid(data) {
    return data.map(x => 1 / (1 + Math.exp(-x)))
}

function softmax(data) { 
    return data.map(x => Math.exp(x) / (data.map(y => Math.exp(y))).reduce((a,b) => a+b)) 
}


-1811852.2913827675
